## Predicting Hospital Readmissions Using Integrated Patient, Clinical, and Socioeconomic Data
 
1.2.1	🎯 Project Objective:
To develop a predictive model for 30-day hospital readmission risk by merging and cleaning patient demographics, clinical encounter data, and socioeconomic data. The goal is to help hospitals reduce readmissions, improve patient outcomes, and reduce costs.
### J. Casey Brookshier
### 7/21/2025

## "Hospital Quality Forecasting: Data-Driven Insights into Readmission Penalties"
Recommended Workflow: Clean First, Then Integrate
## In short: Clean → Standardize → Aggregate → Integrate → Analyze


In [1]:
'''Project Summary: Hospital Quality Forecasting: Data-Driven Insights into Readmission Penalties
You can use this for GitHub README.md, LinkedIn, or your data science portfolio:

🏥 Hospital Quality Forecasting
🎯 Objective
To build a predictive model that estimates hospital readmission performance using publicly available CMS hospital data, infection rates, and area-level deprivation metrics (ADI). The goal is to help stakeholders identify and address risk factors contributing to hospital readmission penalties.

📊 Data Sources
CMS Hospital Readmissions Reduction Program (FY2025): Readmission metrics for multiple clinical conditions.

Healthcare-Associated Infections Data: Facility-level infection control indicators.

Area Deprivation Index (ADI): ZIP+4 level social determinants of health.

🔍 Key Steps
Data Cleaning & Merging: Combined three datasets on hospital IDs and ZIP codes.

Feature Engineering: Created a composite target from six condition-specific readmission ratios.

Modeling: Compared performance of linear regression vs. random forest.

Evaluation: Used RMSE, R², cross-validation, and residual analysis.

Interpretation: Identified top drivers of readmission risk.

Deployment: Built a Streamlit app to make predictions and visualize results.

🧠 Model Results
Random Forest outperformed linear regression with an R² of ~0.75 and RMSE < 0.03

Key predictors: infection control rates, ADI (socioeconomic risk), and condition-specific readmission scores

🚀 Impact
This model can help healthcare administrators, policy analysts, and quality teams proactively identify facilities at risk of CMS readmission penalties, enabling data-informed interventions.


'''

'Project Summary: Hospital Quality Forecasting: Data-Driven Insights into Readmission Penalties\nYou can use this for GitHub README.md, LinkedIn, or your data science portfolio:\n\n🏥 Hospital Quality Forecasting\n🎯 Objective\nTo build a predictive model that estimates hospital readmission performance using publicly available CMS hospital data, infection rates, and area-level deprivation metrics (ADI). The goal is to help stakeholders identify and address risk factors contributing to hospital readmission penalties.\n\n📊 Data Sources\nCMS Hospital Readmissions Reduction Program (FY2025): Readmission metrics for multiple clinical conditions.\n\nHealthcare-Associated Infections Data: Facility-level infection control indicators.\n\nArea Deprivation Index (ADI): ZIP+4 level social determinants of health.\n\n🔍 Key Steps\nData Cleaning & Merging: Combined three datasets on hospital IDs and ZIP codes.\n\nFeature Engineering: Created a composite target from six condition-specific readmission rat

In [32]:
# ============================================================
# Hospital Readmission Forecasting – FINAL NA-SAFE PIPELINE
# ============================================================

import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.impute import SimpleImputer
import pickle

# ============================================================
# CONFIG
# ============================================================

BASE_DIR = Path(
    "/Users/caseybrookshier/Desktop/Data Science job attainment planning/"
    "DS Portfolio github friendly/hospital_readmission_forecasting/data"
)

READMISSIONS_FILE = BASE_DIR / "FY_2025_Hospital_Readmissions_Reduction_Program_Hospital.csv"
INFECTIONS_FILE   = BASE_DIR / "Healthcare_Associated_Infections-Hospital.csv"
ADI_FILE          = BASE_DIR / "CO_2023_ADI_9 Digit Zip Code_v4_0_1.csv"

OUTPUT_DATASET = BASE_DIR / "hospital_readmissions_analytic_table.csv"
ARTIFACT_DIR   = BASE_DIR / "artifacts"

READMISSION_METRICS = [
    "Excess Readmission Ratio",
    "Predicted Readmission Rate",
    "Expected Readmission Rate",
]

LEAKAGE_KEYWORDS = [
    "predicted_readmission_rate",
    "expected_readmission_rate",
]

# ============================================================
# LOAD DATA
# ============================================================

readmissions_df = pd.read_csv(READMISSIONS_FILE)
infections_df   = pd.read_csv(INFECTIONS_FILE)
adi_df          = pd.read_csv(ADI_FILE)

# ============================================================
# CANONICAL KEY NORMALIZATION
# ============================================================

readmissions_df["Facility ID"] = readmissions_df["Facility ID"].astype(str)
infections_df["Facility ID"]   = infections_df["Facility ID"].astype(str)

infections_df["ZIP Code"] = (
    infections_df["ZIP Code"].astype(str).str.zfill(5)
)

print("✅ Canonical IDs enforced")

# ============================================================
# CLEAN READMISSIONS
# ============================================================

readm = readmissions_df.copy()

readm["measure_code"] = (
    readm["Measure Name"]
    .str.replace("READM-30-", "", regex=False)
    .str.replace("-HRRP", "", regex=False)
    .str.lower()
)

for col in READMISSION_METRICS:
    readm[col] = pd.to_numeric(readm[col], errors="coerce")

readm = readm.dropna(subset=["Excess Readmission Ratio"])

readm_wide = readm.pivot_table(
    index=["Facility ID", "Facility Name", "State"],
    columns="measure_code",
    values=READMISSION_METRICS
)

readm_wide.columns = [
    f"{metric.replace(' ', '_').lower()}__{measure}"
    for metric, measure in readm_wide.columns
]

readm_wide = readm_wide.reset_index()

# ============================================================
# CLEAN INFECTIONS
# ============================================================

infect = infections_df.copy()

infect["Score"] = pd.to_numeric(infect["Score"], errors="coerce")
infect = infect.dropna(subset=["Score"])

infect["measure_clean"] = (
    infect["Measure Name"]
    .str.replace("[^A-Za-z0-9]+", "_", regex=True)
    .str.lower()
)

infect_wide = infect.pivot_table(
    index="Facility ID",
    columns="measure_clean",
    values="Score",
    aggfunc="mean"
)

infect_wide = infect_wide.add_prefix("infection__").reset_index()

# ============================================================
# CLEAN ADI
# ============================================================

adi = adi_df.rename(columns={
    "ZIP_4": "zip",
    "ADI_NATRANK": "adi_national",
    "ADI_STATERNK": "adi_state",
})

adi["zip"] = adi["zip"].astype(str).str.zfill(5)
adi["adi_national"] = pd.to_numeric(adi["adi_national"], errors="coerce")
adi["adi_state"] = pd.to_numeric(adi["adi_state"], errors="coerce")

adi = adi.dropna(subset=["adi_national", "adi_state"])

adi_zip = (
    adi.groupby("zip", as_index=False)
       .agg({"adi_national": "mean", "adi_state": "mean"})
)

# ============================================================
# FACILITY → ADI BRIDGE
# ============================================================

facility_zip = (
    infections_df[["Facility ID", "ZIP Code"]]
    .drop_duplicates()
    .rename(columns={"ZIP Code": "zip"})
)

facility_adi = facility_zip.merge(
    adi_zip,
    on="zip",
    how="left"
)

# ============================================================
# FINAL ANALYTIC TABLE
# ============================================================

final_df = (
    readm_wide
    .merge(infect_wide, on="Facility ID", how="left")
    .merge(
        facility_adi[["Facility ID", "adi_national", "adi_state"]],
        on="Facility ID",
        how="left"
    )
)

excess_cols = [
    c for c in final_df.columns
    if c.startswith("excess_readmission_ratio__")
]

final_df["composite_readmission_score"] = final_df[excess_cols].mean(axis=1)

final_df.to_csv(OUTPUT_DATASET, index=False)
print("✅ Analytic dataset saved")

# ============================================================
# MODEL PREP (NA-SAFE)
# ============================================================

drop_cols = (
    ["Facility ID", "Facility Name", "State", "composite_readmission_score"]
    + [c for c in final_df.columns if any(k in c for k in LEAKAGE_KEYWORDS)]
)

X = final_df.drop(columns=drop_cols)
y = final_df["composite_readmission_score"]

# 🔒 Drop all-NaN columns FIRST
X = X.dropna(axis=1, how="all")

# 🔒 Impute remaining missing values
imputer = SimpleImputer(strategy="mean")
X_imputed = pd.DataFrame(
    imputer.fit_transform(X),
    columns=X.columns,
    index=X.index
)

# ============================================================
# TRAIN / TEST
# ============================================================

X_train, X_test, y_train, y_test = train_test_split(
    X_imputed, y, test_size=0.2, random_state=42
)

lr = LinearRegression()
lr.fit(X_train, y_train)

rf = RandomForestRegressor(
    n_estimators=100,
    random_state=42,
    n_jobs=-1
)
rf.fit(X_train, y_train)

def evaluate(model):
    preds = model.predict(X_test)
    return {
        "RMSE": np.sqrt(mean_squared_error(y_test, preds)),
        "R2": r2_score(y_test, preds)
    }

print("\n📊 Model Performance")
print("Linear Regression:", evaluate(lr))
print("Random Forest:", evaluate(rf))

# ============================================================
# SAVE ARTIFACTS
# ============================================================

ARTIFACT_DIR.mkdir(exist_ok=True)

with open(ARTIFACT_DIR / "random_forest_model.pkl", "wb") as f:
    pickle.dump(rf, f)

with open(ARTIFACT_DIR / "feature_names.pkl", "wb") as f:
    pickle.dump(list(X_imputed.columns), f)

print("\n✅ Model artifacts saved")


✅ Canonical IDs enforced
✅ Analytic dataset saved

📊 Model Performance
Linear Regression: {'RMSE': 0.02836671604393265, 'R2': 0.7403724991328297}
Random Forest: {'RMSE': 0.0156257539983864, 'R2': 0.9212202613323425}

✅ Model artifacts saved
